In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
def pickle_read(fileName):
    with open(fileName,'rb') as file_opened:
        return pickle.load(file=file_opened)

def pickle_write(fileName, obj):
    with open(fileName,'wb') as file_opened:
        pickle.dump(obj=obj, file=file_opened)

In [3]:
def save_as_pickle():
    pickle_write(fileName='./processed/obj/tf.pk', obj=tf)
    pickle_write(fileName='./processed/obj/description_matrix.pk', obj=desription_matrix)
    pickle_write(fileName='./processed/obj/cosine_sim.pk', obj=cosine_sim)
    pickle_write(fileName='./processed/obj/movies_df.pk',obj=movies_df)
    pickle_write(fileName='./processed/obj/indices.pk',obj=indices)


In [4]:
columns_movie = ['genres','belongs_to_collection','id','title','popularity','vote_average','vote_count','tagline','overview']
movies_df_ori = pd.read_csv('./data/movies_metadata.csv')
movies_df_ori = movies_df_ori.drop([19730, 29503, 35587])
movies_df_ori['popularity'] = movies_df_ori['popularity'].astype('float')

/home/abdev/projects/mlai-projects/movie-recommendation/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
movies_df = movies_df_ori.loc[ (movies_df_ori['vote_count']>30) & (movies_df_ori['popularity']>4) ].reset_index()
print(movies_df.shape)
# movies_df.describe()

(9262, 25)


# using tags and overview

In [6]:
movies_df['tagline'] = movies_df['tagline'].fillna('')
movies_df['overview'] = movies_df['overview'].fillna('')
movies_df['description'] = movies_df['tagline'] + movies_df['overview']
movies_df['description'] = movies_df['description'].fillna('')

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [8]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')

In [9]:
desription_matrix = tf.fit_transform(movies_df['description'])
desription_matrix.shape

(9262, 261368)

In [10]:
cosine_sim = linear_kernel(desription_matrix, desription_matrix)

In [11]:
movies_df.reset_index()
indices = pd.Series(movies_df.index, index=movies_df['title'])

In [12]:
human_friendly_cols = ['id','title','overview','tagline']

In [13]:
def get_recommendations(title, cosine_sim=cosine_sim, new_data=False):
    idx = 0 if new_data else indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return movies_df.iloc[movie_indices][human_friendly_cols]

In [14]:
get_recommendations('The Dark Knight').head(10)

,id,title,overview,tagline
6191,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,The Legend Ends
83,414,Batman Forever,The Dark Knight of Gotham City confronts a das...,"Courage now, truth always..."
725,364,Batman Returns,"Having defeated the Joker, Batman now faces th...","The Bat, the Cat, the Penguin."
6141,69735,Batman: Year One,Two men come to Gotham City: Bruce Wayne after...,A merciless crime turns a man into an outlaw.
329,268,Batman,The Dark Knight of Gotham City begins his war ...,Have you ever danced with the devil in the pal...
5709,40662,Batman: Under the Red Hood,Batman faces his ultimate challenge as the mys...,Dare to Look Beneath the Hood.
6639,142061,"Batman: The Dark Knight Returns, Part 2",Batman has stopped the reign of terror that Th...,Justice Returns... Vengeance Returns... Redemp...
8995,324849,The Lego Batman Movie,In the irreverent spirit of fun that made “The...,Always be yourself. Unless you can be Batman.
7588,20077,Batman vs Dracula,Gotham City is terrorized not only by recent e...,There's another bat in Gotham...
3842,16234,Batman Beyond: Return of the Joker,"The Joker is back with a vengeance, and Gotham...","This Time, The Joker Is Wild"


In [15]:
# sub_df = movies_df[ movies_df['title'].str.contains('Avengers: Age of Ultron') ]
# sub_df.append( [] )
# movies_df['tagline'] = movies_df['tagline'].fillna('')
# movies_df['overview'] = movies_df['overview'].fillna('')
# movies_df['description'] = movies_df['description'].fillna('')

# sub_df = ('Part of the journey is the end'+
# 'After the devastating events of Avengers: Infinity War, the universe is in ruins due to the efforts of the Mad Titan, Thanos. With the help of remaining allies, the Avengers must assemble once more in order to undo Thanos actions and restore order to the universe once and for all, no matter what consequences may be in store')

sub_df = pd.DataFrame( [['Endgame','Part of the journey is the end','After the devastating events of Avengers: Infinity War, the universe is in ruins due to the efforts of the Mad Titan, Thanos. With the help of remaining allies, the Avengers must assemble once more in order to undo Thanos actions and restore order to the universe once and for all, no matter what consequences may be in store']], columns=['title','tagline','overview'] )
sub_df['description'] = sub_df['tagline'] + sub_df['overview']
sub_df


,title,tagline,overview,description
0,Endgame,Part of the journey is the end,After the devastating events of Avengers: Infi...,Part of the journey is the endAfter the devast...


In [16]:
sub_des_matrix = tf.transform(sub_df['description'])
sub_des_matrix.shape

(1, 261368)

In [17]:
sub_cosine_sim = linear_kernel(sub_des_matrix, desription_matrix)
sub_cosine_sim.shape

(1, 9262)

In [18]:
get_recommendations('Avengers: Age of Ultron', cosine_sim=sub_cosine_sim, new_data=True)

,id,title,overview,tagline
7721,271110,Captain America: Civil War,"Following the events of Age of Ultron, the col...",Divided We Fall
75,27793,The Neverending Story III: Escape from Fantasia,A young boy must restore order when a group of...,It's a brand new beginning in the adventure th...
8097,14613,Next Avengers: Heroes of Tomorrow,The children of the Avengers hone their powers...,The children of heroes past are our only hope ...
8308,340382,Attack on Titan II: End of the World,Eren (Haruma Miura) leaves to restore a break ...,The world is merciless.
6536,64689,Killing Them Softly,Jackie Cogan is an enforcer hired to restore o...,In America you're on your own.
7846,119569,Marvel One-Shot: Item 47,"Benny and Claire, a down on their luck couple ...","The Avengers Won The Battle, But a Few Items W..."
7350,76757,Jupiter Ascending,In a universe where human genetic material is ...,Expand your universe.
6807,192538,Hammer of the Gods,Action epic sees a passionate young man transf...,Blood will Reign
4987,12912,Chaos Theory,"Frank Allen, a professional speaker who lectur...",This man will bring order to the universe...or...
1089,11639,The Dark Crystal,"On another planet in the distant past, a Gelfl...","Another World, Another Time... In the Age of W..."
